In [1]:
import bel_konzept as bk
import pandas as pd
import numpy as np

In [2]:
df_whg = pd.DataFrame(
    [('Schlafen',13.47,'zu'),
     ('Bad',9.06,'ab',45),
     ('Abstellraum',2.57,'ab',25),
     ('Kochen',7.16,'ab',45),
     ('Wohnen',26.98,'zu'),
     ('Zimmer_2',13.96,'zu'),
     ('Zimmer_1',11.89,'zu'),
     ('WC',1.45,'ab',25),
     ('Flur',9.07)
    ],
    columns=['Raum','Flaeche','Typ','dV_min']
)
df_whg

,Raum,Flaeche,Typ,dV_min
0,Schlafen,13.47,zu,NaN
1,Bad,9.06,ab,45.0
2,Abstellraum,2.57,ab,25.0
3,Kochen,7.16,ab,45.0
4,Wohnen,26.98,zu,NaN
5,Zimmer_2,13.96,zu,NaN
6,Zimmer_1,11.89,zu,NaN
7,WC,1.45,ab,25.0
8,Flur,9.07,None,NaN


Nutzfläche in $m^2$

In [3]:
A_N = df_whg.Flaeche.sum()
A_N

95.61000000000001

Gesamter Zuluftvolumenstrom nach DIN 1946-6

In [4]:
dV_ges_zu = bk.dV_ges_zu(A_N)
dV_ges_zu # Gesamter Zuluftvolumenstrom in m**3/h

120.8102279

Gesamter Abluftvolumenstrom nach DIN 1946-6

In [5]:
dV_ges_ab = df_whg[df_whg.Typ == 'ab'].dV_min.sum()
dV_ges_ab # Gesamter Abluftvolumenstrom in m**3/h

140.0

Gesamter Volumenstrom

In [6]:
dV_ges=max(dV_ges_zu,dV_ges_ab)
dV_ges

140.0

Infiltrationsvolumenstrom nach DIN 1946-6

In [7]:
f_inf = 0.4 # Mehrgeschossig, Installationsschacht
n_50 = 1 # h**(-1)
h = 2.50 # m
delta_p = 5 # Mehrgeschossig, Windschwach

dV_inf = bk.dV_inf(f_inf,A_N,h,n_50,delta_p)
dV_inf # Infiltrationsvolumenstrom in m**3/h

20.58274616569973

Lüftungsvolumenstrom nach DIN 1946-6

In [8]:
dV_L = dV_ges - dV_inf
dV_L

119.41725383430027

In [9]:
df_whg[df_whg.Typ=='zu'].Raum

0    Schlafen
4      Wohnen
5    Zimmer_2
6    Zimmer_1
Name: Raum, dtype: object

In [10]:
df_whg['f'] = pd.Series([2,3,2,2],index=[0,4,5,6])
df_whg

,Raum,Flaeche,Typ,dV_min,f
0,Schlafen,13.47,zu,NaN,2.0
1,Bad,9.06,ab,45.0,NaN
2,Abstellraum,2.57,ab,25.0,NaN
3,Kochen,7.16,ab,45.0,NaN
4,Wohnen,26.98,zu,NaN,3.0
5,Zimmer_2,13.96,zu,NaN,2.0
6,Zimmer_1,11.89,zu,NaN,2.0
7,WC,1.45,ab,25.0,NaN
8,Flur,9.07,None,NaN,NaN


In [13]:
f_ges = df_whg.f.sum()
df_whg['dV_zu'] = df_whg.f/f_ges*dV_L
df_whg['dV_ab'] = df_whg.dV_min/df_whg.dV_min.sum()*dV_L
df_whg = df_whg.fillna(0)
df_whg

,Raum,Flaeche,Typ,dV_min,f,dV_zu,dV_ab
0,Schlafen,13.47,zu,0.0,2.0,26.537168,0.000000
1,Bad,9.06,ab,45.0,0.0,0.000000,38.384117
2,Abstellraum,2.57,ab,25.0,0.0,0.000000,21.324510
3,Kochen,7.16,ab,45.0,0.0,0.000000,38.384117
4,Wohnen,26.98,zu,0.0,3.0,39.805751,0.000000
5,Zimmer_2,13.96,zu,0.0,2.0,26.537168,0.000000
6,Zimmer_1,11.89,zu,0.0,2.0,26.537168,0.000000
7,WC,1.45,ab,25.0,0.0,0.000000,21.324510
8,Flur,9.07,0,0.0,0.0,0.000000,0.000000
